# Simulation of a positron-induced Muon Source (part 1)

### Description

The production of a high brillance muon beam is one of the most important challenge for the future of Particle Physics. A particularly interesting idea consists of shooting high energy positrons on a target, aiming at the production of muons by means of the process $e^+ + e^- \rightarrow \mu^+ + \mu^-$. To mimize the divergence of the resulting "muon beam", the positrons energy is chosen so that the reaction occurs close to threshold (assuming the electrons in the target to be at rest). The main goal of this project is to produce a Monte Carlo simulation of such a process. 

### Assignments

The project is split into two parts, which are both reported here for consistency. This group has only to cover _part 1_.

#### Part 1: generate a simulated dataset of $e^+ + e^- \rightarrow \mu^+ + \mu^-$ events.

1. Compute the process leading-order cross section, $\sigma(\theta; \sqrt{s})$, as a function of the scattering angle $\theta$ and with the  center of mass energy $\sqrt{s}$ as a parameter. Start by computing it in the center of mass system. N.B.: textbooks reports such cross section in the relativistic limit, i.e. for $\sqrt{s}\gg m_\mu$, which is clearly not the case here ($\sqrt{s}\sim 2m_\mu$);
2. compute and display the angle and momentum components distributions of the emerging muon pairs;
3. boost muons four-momenta in the laboratory frame, i.e. in the frame where the electron is at rest and the positron has enough energy to give rise to the process;
4. write a Monte Carlo simulation that generates scattering events following the distrubtions that you found analytically; 
5. produce a synthetic dataset of about $N=10^5$ (or more) events. Events should be listed as rows in a file with columns representing the muons coordinates (keep in mind that in the lab frame muons are relativistic and thus the number of coordinates can be only 3 per muon);
6. assume a $3$ cm thick Beryllium block is used as target and a rate of positron on target of $10^6$ Hz. Compute the rescaling factor (weight) you need to apply to the $N$ simulated events such that they represent the statistics that would be gathered in a week of countinuous operations;
7. repeat what done so far simulating now the actual transverse shape and energy spread of the beam: for the former assume a flat distribution in a circle of radius $r=1$ cm and for the latter a gaussian distribution centered at the nominal beam energy and a width of $0.5$ GeV;
8. given that the electrons traversing the target lose energy as $E(z)=E_0 \exp{-z/X_0}$ (with z the longitudinal coordinate of the target, the one parallel to the beam direction and $X_0$ (35.28 cm, Dudo) is the Beryllium radiation length), compute the nominal beam energy $E_0$ such that muon pairs can be generated along the whole length of the target;
9. (optional) take the former point into account when generating the events (i.e. the proccess $\sqrt{s}$ depend on the position along the target where the $e^+ - e^-$ scattering occurrs.

### Simulation of positron-induced Muon Source

The main goal of this project is the simulation of a muon source induced by an electron-positron pair annihilation.
The calculation of the cross section comes from the derivation of the unpolarised scattering amplitude - we are not interested in the polarisation of both incoming and outgoing particles. 
Moreover, we are not interested in considering the annihilation in the usual Ultra Relativistic limit ($m \sim 0$), but we are interested in the range of energy close to the threshold for the muon pair production: $\sqrt s \sim 2m_\mu$. 

The differential cross section for the considered process, in the CoM frame (\*-quantities), has the following expression:
  $$\left(\frac{d\sigma}{d\Omega}\right)^* \Big\rvert_{CoM} (\sqrt s, \theta^*, \phi^*)= \frac{\alpha^2}{4s}\left(1-\frac{4m_\mu^2}{s}\right)^{1/2}\left(1-\frac{4m_e^2}{s}\right)^{-1/2}\left[1+\frac{4}{s}\left(m_e^2+m_\mu^2\right)+\left(1-\frac{4m_\mu^2}{s}\right)\left(1-\frac{4m_e^2}{s}\right)cos^2\theta^*\right]$$

where the differential cross section in the center of mass is expressed as a function of the (CoM) energy $\sqrt s$ (the first Mandelstam variable, equal for both $e$ and $\mu$ pairs in che cms frame), the masses of $e^+$ and $e^-$ $m_e$ and of the muons $m_\mu$, and the emission angle in the center of mass $\theta$. ($\alpha$ is the EM constant, $\alpha=\frac{e^2}{4\pi}$). One can also notice that the differential cross section depends only on the $\theta$ angle, and not on the other angle $\phi$.

The total cross section can be found integrating over the solid angle. 
$$ \sigma (\sqrt s) = \int_{4\pi} d \Omega^* \frac{d\sigma^*}{d\Omega^*} = \int_0^{\pi} d\theta^* sin\theta^* \int_0^{2\pi} d\phi^* \frac{d\sigma^*}{d\Omega^*} =  4\pi\frac{\alpha^2}{3s^3}\left(1-\frac{4m_\mu^2}{s}\right)^{1/2}\left(1-\frac{4m_e^2}{s}\right)^{-1/2}(2m_e^2+s)(2m_\mu^2+s) $$ 
 
The angular distribution can be simply computed normalizing the differential cross section with respect to the total one: 
$$ f(\theta^*, \phi^*) = \frac{1}{\sigma} \frac{d\sigma^*}{d\Omega^*}$$ 

As we are considering a case in which we have a stationary target, we need to pass from the CM coordinates to the LAB ones. Of course a Lorentz boost is going to be involved for both angles and spatial coordinates. 

In the CM frame, the 4-momentum for the incoming and outgoing particles can be written as: 
$$p_{e^\pm}^* = (E^*, \pm \vec p_e^{\,*}) \,\,\,\,\,\,\, p_{\mu ^\pm}^{\prime\,*} = (E^*, \pm \vec p_\mu^{\,\prime\,*})$$

The speed of the center of mass $\beta^*$, in the case in which we consider the electron at rest is given by: 
        $$ \beta^* = \frac{p_{e^+}}{E_{e^+}+m_{e^-}}$$
Moreover, as we consider a process in which there is the production of two muons ($m_\mu \sim 207 m_e$), we can well approximate the parameter $ \beta^* \sim \frac{p_{e^+}}{E_{e^+}} = \beta_{e^+} $

The energy of the CoM $E^*$ can be written in terms of the Mandelstam variable *s*: $\sqrt s = \sqrt{p_{e^-}^2+p_{e^+}^2} = 2E^*$ ( $\sim 2m_\mu$ in the considered case).

And so, it follows:

$$\beta^*_{e^\pm} = \frac{\lvert\vec p^{\,*}\rvert}{E^*} = \frac{ \sqrt{(E^*)^2-m_e^2}}{E^*} = \sqrt{1-\frac{4m_e^2}{s} } $$

And analogously, $$\beta^*_{\mu^\pm} = \sqrt{1-\frac{4m_\mu^2}{s}}$$
(The factors we have in the cross section formula).

We can now observe that the norm of the momentum of the outgoing muon can be written as follows:
$$ \lvert \vec p^{\,*}_\mu \rvert = \sqrt {\frac{s}{4}-m_\mu^2} $$
And the momentum components' distributions can be simply derived knowing the center of mass energy - s - and the emission angles in the CoM - $ \theta^* $ and $ \phi^* $. 

Finally, the relation between the Lab angles and the COM ones has the usual formula: 
$$tan \theta_\mu  = \frac {sin\theta^*_\mu}{\gamma(cos\theta^*_\mu+\beta / \beta^*_\mu)}   $$

If we are consider, without loss of generalities, a boost towards the x axis, the relatoins between the energy and momentum components become:  
$$ E_1 = \gamma ( E^*+\beta p^*) \,\,\,\,\,\, p_1 = \gamma(\beta E^*+p^* ) $$

$$ E_2 = \gamma ( E^*-\beta p^*) \,\,\,\,\,\, p_2 = \gamma(\beta E^*-p^* ) $$



2. Compute and display the angle and momentum components distributions of the emerging muon pairs;

#### Part 2: use the synthetic dataset to design an experiment

Assume a $2$ meter long, $1.7$ Tesla dipole magnet is placed after the target. Assume a number of tracking detectors can be placed before the target, after the target before the magnet (one line) and after the magnet (two lines, one for positive the other for negative muons); those could be made of silicon pixels, with a single-hit resolution varying from 50 to 200 ${\rm \mu m}$.  
Try to address the following items keeping in mind that money matters (i.e. you cannot buy an infinite number of detectors and the smaller the single-hit resolution the higher the cost) and that the layout of the experiment has to be as compact as possible (say whithin 20 m).

1. Define the sizes of the various detectors, such that all the particles traverse them (i.e. $100\%$ detector acceptance); 
2. have each event of the synthetich dataset "interacting" with the detectors, i.e. generate a dataset with the hits (properly smeared due to detector resolutions) left by particles crosssing each detector. That should correspond to the actual measurments gathered by the experiment;
3. add some random noise to such measurements (e.g. for each detector accordingly to poisson statistic of mean about 1);
4. develop an algorithm to track the particles, both the incoming positrons and the outgoing muons.
5. keep in consideration the fact that the two muons emerge synchronously from the target: how small the pitch (the distance between individual pixels) have to be of the detectors prior to the magnet? If that is too small, adjust the tracking algorithm accordingly (i.e. the reconstruct muon tracks could share hits in those detectors) 
6. compute the resolution of the muon momenta and of the $\sqrt(s)$;
7. (optional, a though one..) correct the muon trajectories for the transverse position of the hard scattering on the target section. Actually the incoming positrons are both dispersed in space (the flat distribution with  $r=1$ cm) and divergent; one would need to correct for that too.

### References

* [LEMMA](https://arxiv.org/pdf/1509.04454.pdf) paper. The original paper describing the positron-induced low emittance muon source, ehere all the relevant kinematic features of the process have been studied
* [Babayaga](https://www2.pv.infn.it/~hepcomplex/babayaga.html) event generator. You may want to install and run it as a comparison for your results.
* [2018 Experiment](https://arxiv.org/pdf/1909.13716.pdf): the paper describing the (very poor..) results of the experiment carried out in summer 2018
* [2021 proposal](https://cds.cern.ch/record/2712394?ln=en): the proposal for the experiment in 2021


### Contact

* Marco Zanetti <marco.zanetti@unipd.it>
* Camilla Curatolo <camilla.curatolo@pd.infn.it>
* Jacopo Pazzini <jacopo.pazzini@unipd.it>
* Alberto Zucchetta <alberto.zucchetta@pd.infn.it>